In [ ]:
# Aim: To implement a python program for AdaBoosting

# Step 1: Import Libraries
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Step 2: Load and Prepare Data
df = pd.DataFrame({
    'X1':[1,2,3,4,5,6,6,7,9,9],
    'X2':[5,3,6,8,1,9,5,8,9,2],
    'label':[1,1,0,1,0,1,0,1,0,0]
})

X = df[['X1','X2']].values
y = df['label'].values

# convert y into {-1, +1} form for AdaBoost math
y_transformed = np.where(y==1, 1, -1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_transformed, test_size=0.3, random_state=42
)

# Step 3: Initialize Parameters
n_estimators = 5
n_samples = X_train.shape[0]
weights = np.ones(n_samples) / n_samples
classifiers = []
alphas = []

# Step 4: Train Weak Classifiers
for _ in range(n_estimators):
    clf = DecisionTreeClassifier(max_depth=1)
    clf.fit(X_train, y_train, sample_weight=weights)

    y_pred = clf.predict(X_train)

    # Calculate weighted error
    err = np.sum(weights * (y_pred != y_train)) / np.sum(weights)
    if err == 0:
        err = 1e-10

    # Compute alpha
    alpha = 0.5 * np.log((1 - err) / err)

    # Update weights
    weights = weights * np.exp(-alpha * y_train * y_pred)
    weights /= np.sum(weights)  # normalize

    classifiers.append(clf)
    alphas.append(alpha)

# Step 5: Make Predictions
def predict(X):
    final_score = np.zeros(X.shape[0])
    for clf, alpha in zip(classifiers, alphas):
        preds = clf.predict(X)
        final_score += alpha * preds
    return np.sign(final_score)

y_pred_test = predict(X_test)

# Step 6: Evaluate Model
acc = accuracy_score(y_test, y_pred_test)

# Step 7: Output Results
print("Final Accuracy on Test Set:", acc)
print("Classifier Weights (alphas):", alphas)